# Segmenting and Clustering Neighborhoods in Toronto

## First part: scraping the data from the wikipedia page  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [198]:
import pandas as pd
import numpy as np

wikiFrame=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=wikiFrame[0]
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)

#Only process the cells that have an assigned borough.
df=df[df['Borough']!='Not assigned']

#More than one neighborhood can exist in one postal code area, we group them and separate them by comma. 
df=df.groupby(['PostalCode','Borough'],as_index=False)['Neighbourhood'].apply(lambda x: ','.join(x))
df=df.reset_index().rename(columns={0:'Neighbourhood'})

#cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)




Shape of the dataframe for verification


In [201]:
df.shape


(103, 3)